# **Feed Forward & Back-Propagation Learning Algorithm**  
**Date:** 20/01/25  

**B RITESH KUMAR**  
**Roll Number:** CS22B2043  


# Question 1
Implement the simple neural network algorithm from scratch in Python.
Initialize the weights with [0 0 0] and a learning rate of 0.0001.
For each iteration, calculate the output of the simple neural network for each input in the training set.
Use MSE to computer the error for all samples
Update the weights using the gradient descent procedure.
Repeat the above steps until the simple neural network converges or a maximum number of iterations is reached.
Test the trained simple neural network on a separate test set, explain how you came up with the test set.
Use the step function as an  activation function in the output layer and sigmoid function for other layers.

Use the IRIS Dataset for the above, considering all four features: sepal length, sepal width, petal length, and petal width, but only two classes -  Setosa, and Versicolor.  Drop the feature vectors of the other class.
Please find the dataset here - Iris Dataset


In [112]:
import numpy as np
import pandas as pd

In [113]:
#IMPORTING THE DATASET
data1=pd.read_csv("iris.csv").drop('Id', axis=1)
data1.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [114]:
#Dropping the Iris-verginica column
data1 = data1[data1.Species != 'Iris-virginica']
data1

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,Iris-versicolor
96,5.7,2.9,4.2,1.3,Iris-versicolor
97,6.2,2.9,4.3,1.3,Iris-versicolor
98,5.1,2.5,3.0,1.1,Iris-versicolor


In [115]:
print(data1['Species'].value_counts())

Species
Iris-setosa        50
Iris-versicolor    50
Name: count, dtype: int64


In [116]:
data1 = data1[data1['Species'].isin(['Iris-setosa', 'Iris-versicolor'])]


In [117]:

data1['Species'] = data1['Species'].apply(lambda x: 0 if x == 'Iris-setosa' else 1)

# Prepare input features and labels
X = data1.iloc[:, :-1].values
y = data1.iloc[:, -1].values
# Normalize the features
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
print(X)
print(y)

[[-0.5810659   0.85713543 -1.01435952 -1.03824799]
 [-0.89430898 -0.19845007 -1.01435952 -1.03824799]
 [-1.20755205  0.22378413 -1.08374115 -1.03824799]
 [-1.36417359  0.01266703 -0.94497788 -1.03824799]
 [-0.73768744  1.06825253 -1.01435952 -1.03824799]
 [-0.11120129  1.70160383 -0.80621461 -0.68329141]
 [-1.36417359  0.64601833 -1.01435952 -0.8607697 ]
 [-0.73768744  0.64601833 -0.94497788 -1.03824799]
 [-1.67741667 -0.40956718 -1.01435952 -1.03824799]
 [-0.89430898  0.01266703 -0.94497788 -1.21572628]
 [-0.11120129  1.27936963 -0.94497788 -1.03824799]
 [-1.05093052  0.64601833 -0.87559625 -1.03824799]
 [-1.05093052 -0.19845007 -1.01435952 -1.21572628]
 [-1.8340382  -0.19845007 -1.22250442 -1.21572628]
 [ 0.51528486  1.91272093 -1.15312279 -1.03824799]
 [ 0.35866332  2.75718933 -0.94497788 -0.68329141]
 [-0.11120129  1.70160383 -1.08374115 -0.68329141]
 [-0.5810659   0.85713543 -1.01435952 -0.8607697 ]
 [ 0.35866332  1.49048673 -0.80621461 -0.8607697 ]
 [-0.5810659   1.49048673 -0.94

In [118]:
# Ensure balanced split by class
setosa_indices = np.where(y == 0)[0]
versicolor_indices = np.where(y == 1)[0]

train_size_setosa = int(0.7 * len(setosa_indices))
train_size_versicolor = int(0.7 * len(versicolor_indices))

train_indices = np.concatenate((setosa_indices[:train_size_setosa], versicolor_indices[:train_size_versicolor]))
test_indices = np.concatenate((setosa_indices[train_size_setosa:], versicolor_indices[train_size_versicolor:]))



X_train = X[train_indices]
y_train = y[train_indices]
X_test = X[test_indices]
y_test = y[test_indices]

In [119]:
#Defining the functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def step_function(x):
    return 1 if x >= 0 else 0

def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

**Training**

In [120]:
def train_perceptron(X_train, y_train, learning_rate=0.0001, max_iterations=10000):
    num_samples, num_features = X_train.shape
    weights = np.zeros(num_features)
    bias = 0  # Adding a bias term

    for _ in range(max_iterations):
        total_error = 0
        for i in range(num_samples):
            net_input = np.dot(weights, X_train[i])+bias
            # if yu want to use sigmoid in between for that below uncomment
            #hidden_output = sigmoid(net_input)
            #output = step_function(hidden_output)
            output = step_function(net_input)
            error = y_train[i] - output
            total_error += abs(error)
            weights += learning_rate * error * X_train[i]
            bias += learning_rate * error
        if total_error == 0:
            break
    return weights,bias

In [121]:
def predict(X_test, weights,bias):
    num_samples = X_test.shape[0]
    predictions = []
    for i in range(num_samples):
        net_input = np.dot(weights, X_test[i])+bias
        #hidden_output = sigmoid(net_input)
        predictions.append(step_function(net_input))
    return np.array(predictions)

In [122]:
trained_weights, trained_bias = train_perceptron(X_train, y_train)

# Predict and evaluate
y_pred = predict(X_test, trained_weights, trained_bias)
mse = mean_squared_error(y_test, y_pred)

print("Trained Weights:", trained_weights)
print("Trained Bias:", trained_bias)
print("Mean Squared Error on Test Set:", mse)


Trained Weights: [ 5.81065904e-05 -8.57135429e-05  1.01435952e-04  1.03824799e-04]
Trained Bias: -0.0001
Mean Squared Error on Test Set: 0.0


# Question 2
Implement the feedforward and backpropagation learning algorithm for multi layer perceptrons in Python for the question provided in the attached image.
Use  the weights and biases as given.
Implement the forward pass.
Compute the loss between the predicted output and the actual output using an appropriate loss function (MSE).
Compute the gradients of the loss function with respect to the weights and biases using the chain rule.
Update the weights and biases.
Iterate over multiple times (epochs), performing forward propagation, loss calculation, backpropagation, and parameter updates in each iteration till convergence (the actual output is the same as the target output).


In [2]:
import numpy as np

In [3]:
# Initialize weights and biases
weights_1 = np.array([[0.5, 0.8], [1.5, 0.2], [0.8, -1.6]])
biases_1 = np.array([[0.7, 1.2]])
weights_2 = np.array([[0.9, 1.2], [-1.7, 2.1]])
biases_2 = np.array([[1.6, -0.2]])

# Input data and true target
inputs = np.array([[1, 0.7, 1.2]])  # Input with bias term included
y_true = np.array([[1, 0]])  # Target output

# Training parameters
epochs = 10000
learning_rate = 0.3  # Selected as a moderate value for stability and speed

**Required Functions**

In [4]:
# Sigmoid function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

# Mean Squared Error (MSE) loss function
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Forward pass
def forward_pass(inputs, weights, biases):
    layer_inputs = inputs @ weights + biases
    layer_outputs = sigmoid(layer_inputs)
    return layer_outputs, layer_inputs

# Backpropagation
def backpropagation(inputs, weights, biases, y_true, y_pred, layer_inputs):
    loss_grad = -(y_true - y_pred)
    activation_grad = loss_grad * sigmoid_derivative(layer_inputs)
    weights_grad = inputs.T @ activation_grad
    biases_grad = np.sum(activation_grad, axis=0, keepdims=True)
    inputs_grad = activation_grad @ weights.T
    return weights_grad, biases_grad, inputs_grad

# Training the model
def train(inputs, y_true, weights_1, biases_1, weights_2, biases_2, epochs, learning_rate):
    for epoch in range(epochs):
        # Forward pass through both layers
        outputs_1, inputs_1 = forward_pass(inputs, weights_1, biases_1)
        outputs_2, inputs_2 = forward_pass(outputs_1, weights_2, biases_2)

        # Compute loss
        loss = mse_loss(y_true, outputs_2)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")

        # Check for convergence
        if np.allclose(outputs_2, y_true, atol=1e-3):
            print("Convergence reached.")
            break

        # Backpropagation for output layer
        weights_2_grad, biases_2_grad, layer_1_outputs_grad = backpropagation(
            outputs_1, weights_2, biases_2, y_true, outputs_2, inputs_2
        )

        # Backpropagation for hidden layer
        weights_1_grad, biases_1_grad, _ = backpropagation(
            inputs, weights_1, biases_1, layer_1_outputs_grad, outputs_1, inputs_1
        )

        # Update weights and biases
        weights_2 -= learning_rate * weights_2_grad
        biases_2 -= learning_rate * biases_2_grad
        weights_1 -= learning_rate * weights_1_grad
        biases_1 -= learning_rate * biases_1_grad

    # Final outputs after training
    return weights_1, biases_1, weights_2, biases_2

# Step function for final predictions
def step_function(x):
    return np.where(x >= 0.5, 1, 0)

In [5]:
# Train the model
weights_1, biases_1, weights_2, biases_2 = train(
    inputs, y_true, weights_1, biases_1, weights_2, biases_2, epochs, learning_rate
)

# Final prediction
outputs_1, _ = forward_pass(inputs, weights_1, biases_1)
outputs_2, _ = forward_pass(outputs_1, weights_2, biases_2)
final_outputs = step_function(outputs_2)

# Print results
print("\nFinal Results:")
print(f"Updated Weights Input to Hidden:\n{weights_1}")
print(f"Updated Biases Input to Hidden:\n{biases_1}")
print(f"Updated Weights Hidden to Output:\n{weights_2}")
print(f"Updated Biases Hidden to Output:\n{biases_2}")
print(f"Final Loss: {mse_loss(y_true, outputs_2)}")
print(f"Predicted Output: {final_outputs}")
print(f"True Output: {y_true}")

Streaming output truncated to the last 5000 lines.
Epoch 5017/10000, Loss: 0.00034579723662079166
Epoch 5018/10000, Loss: 0.00034572735288399307
Epoch 5019/10000, Loss: 0.00034565749722143533
Epoch 5020/10000, Loss: 0.00034558766961624175
Epoch 5021/10000, Loss: 0.0003455178700515669
Epoch 5022/10000, Loss: 0.0003454480985105643
Epoch 5023/10000, Loss: 0.0003453783549764077
Epoch 5024/10000, Loss: 0.0003453086394322912
Epoch 5025/10000, Loss: 0.0003452389518614037
Epoch 5026/10000, Loss: 0.00034516929224696967
Epoch 5027/10000, Loss: 0.0003450996605722079
Epoch 5028/10000, Loss: 0.00034503005682036414
Epoch 5029/10000, Loss: 0.0003449604809746893
Epoch 5030/10000, Loss: 0.0003448909330184527
Epoch 5031/10000, Loss: 0.0003448214129349327
Epoch 5032/10000, Loss: 0.0003447519207074246
Epoch 5033/10000, Loss: 0.0003446824563192348
Epoch 5034/10000, Loss: 0.00034461301975368373
Epoch 5035/10000, Loss: 0.0003445436109941077
Epoch 5036/10000, Loss: 0.00034447423002384654
Epoch 5037/10000, Los